In [48]:
import os
import re
import json
import pandas as pd
import numpy as np

In [37]:

# Reading in the data and visualizing it in an accessible manner
df = pd.read_json('merged_file.json')
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", 80)  # or None to see full text

df.head()

,id,text,created_at,vote_total,comment_count,alias,group_id,index_code,controversial_flag
0,1pl1fvp,Hiker rescued after being stuck in quicksand for hours amid freezing temps i...,2025-12-12 19:54:28+00:00,152,35,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,HPqSy4as,0
1,1pkzill,"EU agrees to indefinitely freeze Russian assets, removing obstacle to Ukrain...",2025-12-12 18:36:28+00:00,1550,43,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,ivJCOQ3f,0
2,1pkyuq1,Health Experts Slam Possible FDA ‘Black Box’ Warning for COVID Vaccines,2025-12-12 18:10:19+00:00,510,62,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,PdAmBfSd,0
3,1pkypnv,"Iran arrests Nobel Peace Prize laureate Narges Mohammadi, supporters say",2025-12-12 18:04:43+00:00,217,9,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,xImO7S8o,0
4,1pkyj4z,Fired Michigan football coach Sherrone Moore is charged with three crimes,2025-12-12 17:57:57+00:00,350,38,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,lGXpOMcU,0


In [38]:
# Cleaning data
df = df[df[LABEL_COL].notna() & df[TEXT_COL].notna()]

df.head()

def preprocess_text(t: str) -> str:
    t = t.lower()
    
    # remove URLs
    t = re.sub(r"http\S+|www\S+", "", t)
    
    # remove user mentions (Reddit / Twitter style)
    t = re.sub(r"u\/\w+|@\w+", "", t)
    
    # remove extra whitespace
    t = re.sub(r"\s+", " ", t).strip()
    
    return t

TEXT_COL = "text"
df["text_clean"] = df[TEXT_COL].apply(preprocess_text)

df.head()



,id,text,created_at,vote_total,comment_count,alias,group_id,index_code,controversial_flag,text_clean
0,1pl1fvp,Hiker rescued after being stuck in quicksand for hours amid freezing temps i...,2025-12-12 19:54:28+00:00,152,35,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,HPqSy4as,0,hiker rescued after being stuck in quicksand for hours amid freezing temps i...
1,1pkzill,"EU agrees to indefinitely freeze Russian assets, removing obstacle to Ukrain...",2025-12-12 18:36:28+00:00,1550,43,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,ivJCOQ3f,0,"eu agrees to indefinitely freeze russian assets, removing obstacle to ukrain..."
2,1pkyuq1,Health Experts Slam Possible FDA ‘Black Box’ Warning for COVID Vaccines,2025-12-12 18:10:19+00:00,510,62,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,PdAmBfSd,0,health experts slam possible fda ‘black box’ warning for covid vaccines
3,1pkypnv,"Iran arrests Nobel Peace Prize laureate Narges Mohammadi, supporters say",2025-12-12 18:04:43+00:00,217,9,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,xImO7S8o,0,"iran arrests nobel peace prize laureate narges mohammadi, supporters say"
4,1pkyj4z,Fired Michigan football coach Sherrone Moore is charged with three crimes,2025-12-12 17:57:57+00:00,350,38,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,lGXpOMcU,0,fired michigan football coach sherrone moore is charged with three crimes


In [41]:
REPLY_COL = "comment_count"
UPVOTE_COL = "vote_total"
df["comment_upvote_ratio"] = df[REPLY_COL] / (df[UPVOTE_COL] + 1)

# Adding a scaled comment to upvote ratio 
df.head()

,id,text,created_at,vote_total,comment_count,alias,group_id,index_code,controversial_flag,text_clean,comment_upvote_ratio
0,1pl1fvp,Hiker rescued after being stuck in quicksand for hours amid freezing temps i...,2025-12-12 19:54:28+00:00,152,35,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,HPqSy4as,0,hiker rescued after being stuck in quicksand for hours amid freezing temps i...,0.228758
1,1pkzill,"EU agrees to indefinitely freeze Russian assets, removing obstacle to Ukrain...",2025-12-12 18:36:28+00:00,1550,43,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,ivJCOQ3f,0,"eu agrees to indefinitely freeze russian assets, removing obstacle to ukrain...",0.027724
2,1pkyuq1,Health Experts Slam Possible FDA ‘Black Box’ Warning for COVID Vaccines,2025-12-12 18:10:19+00:00,510,62,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,PdAmBfSd,0,health experts slam possible fda ‘black box’ warning for covid vaccines,0.121331
3,1pkypnv,"Iran arrests Nobel Peace Prize laureate Narges Mohammadi, supporters say",2025-12-12 18:04:43+00:00,217,9,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,xImO7S8o,0,"iran arrests nobel peace prize laureate narges mohammadi, supporters say",0.041284
4,1pkyj4z,Fired Michigan football coach Sherrone Moore is charged with three crimes,2025-12-12 17:57:57+00:00,350,38,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,lGXpOMcU,0,fired michigan football coach sherrone moore is charged with three crimes,0.108262


In [45]:
# Adding sentiment analysis
from textblob import TextBlob
df["sentiment"] = df["text"].apply(lambda x:TextBlob(x).sentiment.polarity)
df.head()

,id,text,created_at,vote_total,comment_count,alias,group_id,index_code,controversial_flag,text_clean,comment_upvote_ratio,sentiment
0,1pl1fvp,Hiker rescued after being stuck in quicksand for hours amid freezing temps i...,2025-12-12 19:54:28+00:00,152,35,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,HPqSy4as,0,hiker rescued after being stuck in quicksand for hours amid freezing temps i...,0.228758,0.000000
1,1pkzill,"EU agrees to indefinitely freeze Russian assets, removing obstacle to Ukrain...",2025-12-12 18:36:28+00:00,1550,43,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,ivJCOQ3f,0,"eu agrees to indefinitely freeze russian assets, removing obstacle to ukrain...",0.027724,0.000000
2,1pkyuq1,Health Experts Slam Possible FDA ‘Black Box’ Warning for COVID Vaccines,2025-12-12 18:10:19+00:00,510,62,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,PdAmBfSd,0,health experts slam possible fda ‘black box’ warning for covid vaccines,0.121331,-0.083333
3,1pkypnv,"Iran arrests Nobel Peace Prize laureate Narges Mohammadi, supporters say",2025-12-12 18:04:43+00:00,217,9,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,xImO7S8o,0,"iran arrests nobel peace prize laureate narges mohammadi, supporters say",0.041284,0.000000
4,1pkyj4z,Fired Michigan football coach Sherrone Moore is charged with three crimes,2025-12-12 17:57:57+00:00,350,38,Anonymous,0dab5baa-0931-4d9c-bbdf-0de758fca36f,lGXpOMcU,0,fired michigan football coach sherrone moore is charged with three crimes,0.108262,0.000000


In [51]:
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/DAVIDSON/eldevulapally/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [53]:
def vader_features(text):
    scores = vader.polarity_scores(text)
    return pd.Series({
        "vader_neg": scores["neg"],
        "vader_neu": scores["neu"],
        "vader_pos": scores["pos"],
        "vader_compound": scores["compound"],
    })

vader_df = df["text_clean"].apply(vader_features)
df = pd.concat([df, vader_df], axis=1)



In [55]:
# Adding conflict word count
CONFLICT_WORDS = [
    "but", "however", "actually", "wrong", "disagree",
    "no", "not", "never", "false"
]

def conflict_features(text):
    tokens = text.split()
    return pd.Series({
        "conflict_count": sum(t in CONFLICT_WORDS for t in tokens),
        "has_conflict": int(any(t in CONFLICT_WORDS for t in tokens)),
        "exclamations": text.count("!"),
        "questions": text.count("?"),
        "all_caps_ratio": sum(w.isupper() for w in tokens) / (len(tokens) + 1)
    })

conflict_df = df["text_clean"].apply(conflict_features)
df = pd.concat([df, conflict_df], axis=1)


In [57]:
df["abs_vader_compound"] = df["vader_compound"].abs()
df["abs_sentiment"] = df["sentiment"].abs()  # TextBlob polarity


In [58]:
DISAGREE_WORDS = [
    "disagree", "wrong", "false", "misleading",
    "no", "not", "never", "nonsense", "ridiculous"
]

def disagreement_features(text):
    tokens = text.split()
    count = sum(w in tokens for w in DISAGREE_WORDS)
    return pd.Series({
        "disagree_count": count,
        "has_disagree": int(count > 0)
    })

disagree_df = df["text_clean"].apply(disagreement_features)
df = pd.concat([df, disagree_df], axis=1)


In [63]:
df["hour"] = df["created_at"].dt.hour
df["day_of_week"] = df["created_at"].dt.dayofweek
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

In [ ]:
def pronoun_and_length_features(text):
    tokens = text.split()
    n_tokens = len(tokens)

    first_person_count = sum(t in FIRST_PERSON for t in tokens)
    second_person_count = sum(t in SECOND_PERSON for t in tokens)

    return pd.Series({
        "post_length": n_tokens,
        "first_person_count": first_person_count,
        "second_person_count": second_person_count,
        "first_person_ratio": first_person_count / (n_tokens + 1),
        "second_person_ratio": second_person_count / (n_tokens + 1),
    })
pronoun_df = df["text_clean"].apply(pronoun_and_length_features)
df = pd.concat([df, pronoun_df], axis=1)


In [ ]:
# Pass in as CSV file
df.to_csv("reddit_df_more_features.csv", index=False)